In [17]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification
from pathlib import Path
from dotenv import dotenv_values, load_dotenv

try:
    local_config = dotenv_values(".env")
except FileNotFoundError:
    local_config = dict()
load_dotenv()

True

In [18]:
checkpoint_path = Path(local_config['PREDICTION_CHECKPOINT_PATH'])
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, return_tensors='pt')

In [14]:
from plotkarzyna.decode import *
from plotkarzyna.model import predict

In [15]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
    3: "E",
    4: "S"
}
label2id = {
    "O": 0,
    "B": 1,
    "I": 2,
    "E": 3,
    "S": 4,
}

In [16]:
text = "Ala ma kota, który wszedł na drzewo."
# text = "Pogoda pod koniec tygodnia nie będzie miała dla nas litości. W dalszym ciągu spodziewamy się upałów. Miejscami temperatura zdecydowanie przekroczy 30 st. C i tylko lokalnie orzeźwienie przyniosą burze."
text = "Koalicja Obywatelska już odkryła karty i wiemy, jakie nazwiska znajdą się na listach w poszczególnych okręgach podczas jesiennych wyborów do Sejmu. Teraz czas na Lewicę. Partia ujawnia swoich kandydatów."
text = """Jeżeli pole to jest polem centralnym, zależnym tylko od odległości cząstek, a niezależnym od kierunku w przestrzeni, to teoria kwantowa przewiduje, że stan o najniższej energii winien mieć zerowy moment pędu i symetrię sferyczną. Tymczasem pomiary rozkładu ładunku elektrycznego w deuteronie wskazują na wyraźne, jakkolwiek niewielkie odstępstwa od takiej kulistej symetrii. Co więcej, nie można było pogodzić własności magnetycznych deuteronu z własnościami magnetycznymi jego składników. Ponieważ spin deuteronu wynosi 1, więc połówkowe spiny neutronu i protonu muszą być w nim ustawione równolegle. Kierunki wektorów momentów magnetycznych związane są z kierunkami spinów, zatem moment magnetyczny deuteronu winien być sumą momentów protonu i neutronu. Zaobserwowane odstępstwa świadczą o tym, że pewien przyczynek do momentu magnetycznego deuteronu musi pochodzić z ruchu protonu po orbicie z momentem pędu większym od zera. Ponieważ w przypadku sił centralnych stan o najniższej energii musi mieć moment pędu równy zeru, możemy wyciągnąć stąd wniosek o niecentralności sił jądrowych. Muszą one, poza zależnością od odległości nukleonów, zależeć w jakiś sposób od kierunku w przestrzeni. Skąd jednak taka zależność może pochodzić?
 Oddziaływania między nukleonami mogą zależeć również od innych poza ich odległością cech nukleonów, takich jak pęd czy moment pędu w ich ruchu, a także ich spiny i izospiny. Jest oczywiste, że mając do dyspozycji obok wektora odległości R również wektory spinów nukleonów S1 i S2, możemy część niecentralną oddziaływania utworzyć jako zależną od wzajemnej orientacji tych wektorów (rys.4.4). Skąd jednak znaleźć możemy postać tej zależności?
"""
text = """
Nie zawsze jednak to wystarcza.

Jest jeszcze jeden sposób, by ocalić chwasty.
ak naprawdę może się do tego przyczynić każdy, kto ma ogród lub działkę,
kawałek ziemi. I nie chodzi mi tu wcale o pozostawianie chwastów na grządkach,
chociaż w każdym ogrodzie powinien być choć jeden mały zakątek,
w którym nic nie robimy – wtedy w ogrodzie będzie na pewno więcej ptaków i motyli.
Wiele chwastów polnych to piękne, barwne kwiaty, świetnie nadające się na rośliny ozdobne.
Pomysł ten może wydawać się z początku absurdalny, ale... może warto wydzielić fragment swojego ogrodu
i założyć ogródek chwastów? Nasiona można pozbierać latem na polu lub kupić w internecie specjalną
mieszankę – są już dostępne. Najlepiej zmieszać je z nasionami zbóż albo innych roślin uprawnych, 
np. pięknie, błękitnie kwitnącego lnu. Czerwone maki, błękitne chabry, różowe kąkole, fioletowe 
ostróżeczki, pnące się po płotach i krzewach wyki, pełzający po ziemi kurzyślad o uroczych czerwonych 
kwiatkach, maleńkie, kremowe fiołki polne wśród złocistych kłosów jęczmienia mogą stać się oryginalną
ozdobą naszego ogrodu. Na pewno zadziwią i zachwycą przyjaciół, rodzinę, sąsiadów.
"""
# text = """
# Tak naprawdę może się do tego przyczynić każdy, kto ma ogród lub działkę,
# kawałek ziemi. I nie chodzi mi tu wcale o pozostawianie chwastów na grządkach,
# chociaż w każdym ogrodzie powinien być choć jeden mały zakątek,
# w którym nic nie robimy – wtedy w ogrodzie będzie na pewno więcej ptaków i motyli.
# """
# text = """
# Zmiany w środowisku, a także postęp w rolnictwie i nieustanne powtarzanie błędów Zachodu, 
# gdzie obecnie na odtworzenie ekosystemów rolniczych wydaje się ogromne sumy, sprawiają,
# że chwasty giną coraz szybciej.
# """
tokenized = tokenizer([text], return_tensors='pt')
pred = model(
    **tokenized
).logits.argmax(-1)[0]

print(' '.join([
    f"|{tokenizer.decode(tok, clean_up_tokenization_spaces=False)}| ({id2label[int(el)]})" for el, tok in zip(pred, list(tokenized['input_ids'][0]))
]))
tokens = [tokenizer.decode(tok) for tok in list(tokenized['input_ids'][0])]
labels = [id2label[int(el)] for el, tok in zip(
    pred, list(tokenized['input_ids'][0]))]
tokens, labels, aligned_span_inds, spans = predict(text, model, tokenizer, verbose=False)
for span in sorted(spans):
    print(span)

|<s>| (O) |Nie| (O) |zawsze| (O) |jednak| (O) |to| (S) |wystarcza| (O) |.| (O) |Zmiany| (B) |w| (I) |środowisku| (S) |,| (I) |a| (I) |także| (I) |postęp| (B) |w| (I) |rolnictwie| (S) |i| (I) |nieustan| (B) |ne| (I) |powtarz| (I) |anie| (I) |błędów| (B) |Zachodu| (S) |,| (I) |gdzie| (I) |obecnie| (I) |na| (I) |od| (B) |tworzenie| (B) |eko| (B) |systemów| (I) |rolniczych| (E) |wydaje| (I) |się| (I) |ogromne| (B) |sumy| (E) |,| (O) |sprawiają| (O) |,| (O) |że| (O) |chwa| (S) |sty| (S) |giną| (I) |coraz| (I) |szybciej| (E) |.| (O) |Czasem| (O) |jest| (O) |to| (S) |spowodowane| (O) |tym| (B) |,| (I) |że| (I) |niektóre| (B) |gatunki| (I) |roślin| (S) |wycof| (I) |uje| (I) |się| (I) |z| (I) |uprawy| (S) |–| (O) |coraz| (B) |rzadziej| (I) |uprawia| (I) |się| (I) |len| (B) |albo| (I) |gry| (S) |kę| (S) |,| (O) |nie| (O) |mówiąc| (O) |o| (O) |takich| (B) |“| (I) |dzi| (I) |wad| (I) |łach| (I) |”| (I) |jak| (I) |l| (B) |nicz| (S) |nik| (B) |,| (I) |niegdyś| (I) |podstawowe| (I) |źródło| (I) |olej